# Python Client `s2orc-doc2json`

In [2]:
list_pdfs = os.listdir("/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/pdf") 
list_df_tei = os.listdir("/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/pdf_tei_xml") 
list_pdf_json = os.listdir("/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/pdf_json") 
list_pdf_txt = os.listdir("/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/paperwithcode/pdf_txt") 

# Stats in Paper

In [75]:
# imports
import ipdb, os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

## IBM

In [3]:
IBM_train_csv = "~/Research/task-dataset-metric-nli-extraction/data/ibm/exp/few-shot-setup/NLP-TDMS/paperVersion/train.tsv"
IBM_test_csv = "~/Research/task-dataset-metric-nli-extraction/data/ibm/exp/few-shot-setup/NLP-TDMS/paperVersion/test.tsv"

In [4]:
train_IBM = pd.read_csv(IBM_train_csv, 
                    sep="\t", names=["label", "title", "TDM", "Context"])
# train_IBM['label'] = train_IBM.label.apply(lambda x: "true" if x else "false")

test_IBM = pd.read_csv(IBM_test_csv, 
                    sep="\t", names=["label", "title", "TDM", "Context"])

In [186]:
def get_stats(path_to_df): 
    
    unique_labels = path_to_df[(path_to_df.label == True)].TDM.tolist()
    
    TDM = set()
    Uniq_task = set()
    Uniq_dataset = set()
    Uniq_metric = set()
    unknown_count = 0
    avg_tdm_per_paper = defaultdict(lambda : 0)
    
    for contrib in unique_labels:
        split = contrib.split(';')
        
        if(len(split) == 1):
            
            unknown_count += 1 
        else:
            if len(split) !=3:
#                 ipdb.set_trace()
                task, dataset, metric, _ = split
                
            else:
                task, dataset, metric = split
            
            t, d, m = task.strip(), dataset.strip(), metric.strip()
            TDM.add(f"{t}#{d}#{m}")
            
            Uniq_task.add(t)
            Uniq_dataset.add(d)
            Uniq_metric.add(m)
    
    for paper in path_to_df[(path_to_df.label == True) & (path_to_df.TDM != 'unknown') ].title.tolist():
        avg_tdm_per_paper[paper] += 1
    
    print(f"Number of papers: {len(set(path_to_df[(path_to_df.label == True)].title.tolist()))}")
    print(f"Unknown count: {unknown_count}")
    print(f"Total leaderboards: {len(path_to_df[(path_to_df.label == True) & (path_to_df.TDM != 'unknown')].title.tolist())}")
    print(f"Avg leaderboard per paper: {round(np.mean(list(avg_tdm_per_paper.values())), 2)}")
    print(f"Distinc leaderboard: {len(TDM)}")
    print(f"Distinct taks: {len(Uniq_task)}")
    print(f"Distinc datasets: {len(Uniq_dataset)}")
    print(f"Distinc metrics: {len(Uniq_metric)}")
    
    
    return avg_tdm_per_paper

### Train

In [187]:
train_IBM.head()

,label,title,TDM,Context
0,True,D16-1036.pdf,unknown,Multi-view Response Selection for Human-Comput...
1,False,D16-1036.pdf,question answering; SQuAD; F1,Multi-view Response Selection for Human-Comput...
2,False,D16-1036.pdf,relation prediction; FB15K-237; H@1,Multi-view Response Selection for Human-Comput...
3,False,D16-1036.pdf,word sense disambiguation; SemEval 2013; F1,Multi-view Response Selection for Human-Comput...
4,False,D16-1036.pdf,language modeling; 1B Words / Google Billion W...,Multi-view Response Selection for Human-Comput...


In [188]:
train_IBM.title.nunique()

170

In [189]:
avg_tdm_per_paper = get_stats(train_IBM)

Number of papers: 170
Unknown count: 46
Total leaderboards: 327
Avg leaderboard per paper: 2.64
Distinc leaderboard: 78
Distinct taks: 18
Distinc datasets: 44
Distinc metrics: 31


### Test

In [190]:
test_IBM.head()

,label,title,TDM,Context
0,True,1803.11175.pdf,sentiment analysis; SUBJ; Accuracy,Universal Sentence Encoder We present models f...
1,True,1803.11175.pdf,text classification; TREC; Error,Universal Sentence Encoder We present models f...
2,False,1803.11175.pdf,question answering; SQuAD; F1,Universal Sentence Encoder We present models f...
3,False,1803.11175.pdf,relation prediction; FB15K-237; H@1,Universal Sentence Encoder We present models f...
4,False,1803.11175.pdf,word sense disambiguation; SemEval 2013; F1,Universal Sentence Encoder We present models f...


In [191]:
metric = get_stats(test_IBM)

Number of papers: 167
Unknown count: 45
Total leaderboards: 294
Avg leaderboard per paper: 2.41
Distinc leaderboard: 78
Distinct taks: 18
Distinc datasets: 44
Distinc metrics: 31


In [192]:
# Make sure that all leaderboard in test are present in train 
count = []
for paper in test_IBM.TDM.to_list():
    if paper not in train_IBM.TDM.to_list():
        count.append(paper)
print(count)

[]


In [193]:
count = []
for paper in train_IBM.TDM.to_list():
    if paper not in test_IBM.TDM.to_list():
        print(paper)
        count.append(paper)
print(count)

[]


### our dataset

In [194]:
New_train_csv = "~/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_800/twofoldwithunk/fold1/train.tsv"
New_test_csv = "~/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_800/twofoldwithunk/fold1/dev.tsv"

In [195]:
train_New = pd.read_csv(New_train_csv, 
                    sep="\t", names=["label", "title", "TDM", "Context"])
# train_IBM['label'] = train_IBM.label.apply(lambda x: "true" if x else "false")

test_New = pd.read_csv(New_test_csv, 
                    sep="\t", names=["label", "title", "TDM", "Context"])

In [196]:
avg_tdm_per_paper = get_stats(train_New)

Number of papers: 3181
Unknown count: 531
Total leaderboards: 35752
Avg leaderboard per paper: 11.72
Distinc leaderboard: 2518
Distinct taks: 365
Distinc datasets: 1220
Distinc metrics: 733


In [197]:
round(np.sum(list(avg_tdm_per_paper.values())), 2)

35752

In [198]:
avg_tdm_per_paper = get_stats(test_New)

Number of papers: 1361
Unknown count: 269
Total leaderboards: 16700
Avg leaderboard per paper: 12.85
Distinc leaderboard: 1992
Distinct taks: 304
Distinc datasets: 974
Distinc metrics: 561


In [199]:
count = []
for tdm in test_New.TDM.to_list():
    if tdm not in train_New.TDM.to_list():
        count.append(tdm)
print(count)

[]


In [200]:
count = []
for tdm in train_New.TDM.to_list():
    if tdm not in test_New.TDM.to_list():
        count.append(tdm)
print(len(count))

41
